<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-16 11:21:07
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.42 C
-------------------
Today PnL: -20.86 K (-0.15%)
Current PnL: -32.69 L (-20.44%)
CY Booked + Current PnL: -17.01 L (-10.63%)
-------------------
Total profit:  1.18 L
Total loss:  -33.88 L
-------------------
Total Booked + Current PnL: 9.97 L (7.49%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.06%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 98.06 L (69.18%)
Deployed:  1.33 C
Current:  1.42 C
CAGR/XIRR %: 3.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.86,0.89,20.50,21.57,20264.0,870.0,98850.0,397.04,-12.83,56.0,H-SC,3.97,167.0,0.04,0.72,44.25,XR,ATH,FINANCE
18,COALINDIA,2.86,3.44,15.26,19.23,23751.0,5180.0,155640.0,484.83,24.82,49.0,L-LC,11.35,185.0,0.22,1.14,23.40,XY25,ATH,MINING
77,TTKPRESTIG,-0.94,-27.39,37.86,0.09,27702.0,-27607.0,73170.0,770.00,69.90,39.0,M-SC,8.94,250.0,-1.00,0.54,0.70,OX40N,NTT,DURABLES
51,MEDANTA,0.40,-4.51,29.94,24.08,36294.0,-5728.0,121222.0,1486.00,-5.20,53.0,X-SC,11.65,87.0,-0.16,0.89,11.03,XY24,NTT,HEALTHCARE
36,ICICIGI,-0.32,1.17,17.96,19.34,36360.0,2340.0,202449.0,2252.93,-14.93,55.0,X-MC,3.17,66.0,0.06,1.48,17.29,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,11.80,-59.22,257.24,45.70,185501.0,-104703.0,72112.0,26.40,-25.12,63.0,M-SC,26.32,204.0,-0.56,0.53,18.81,XY24,NTT,TRAVEL
30,HAPPSTMNDS,3.76,-51.82,283.70,84.85,175168.0,-66421.0,61744.0,1480.71,-42.72,37.0,H-SC,25.42,152.0,-0.38,0.45,3.76,AR,ATH,IT
23,DIXON,3.09,-18.39,60.85,31.26,114593.0,-42448.0,188320.0,18931.72,-63.52,55.0,X-MC,26.39,54.0,-0.37,1.38,15.57,X40N,ATH,IT
18,COALINDIA,2.86,3.44,15.26,19.23,23751.0,5180.0,155640.0,484.83,24.82,49.0,L-LC,11.35,185.0,0.22,1.14,23.40,XY25,ATH,MINING
19,COFFEEDAY,2.20,-42.68,135.99,35.27,88513.0,-48461.0,65088.0,80.00,-55.73,57.0,L-SC,14.43,266.0,-0.55,0.48,59.30,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,-5.77,-47.62,232.49,74.15,199230.0,-77911.0,85694.0,838.00,-254.09,45.0,H-SC,24.47,145.0,-0.39,0.63,9.85,XR,NTT,CHEMICALS
5,ANGELONE,-4.26,5.79,17.41,24.20,56219.0,17662.0,322912.0,3033.00,53.38,49.0,X-SC,3.65,97.0,0.31,2.36,35.24,X40N,NTT,FINANCE
55,RAJESHEXPO,-4.11,-67.04,203.63,0.07,92059.0,-91968.0,45209.0,518.00,1556.57,46.0,L-SC,8.92,268.0,-1.00,0.33,12.44,OX40N,NTT,JEWELLERY
69,SURYODAY,-3.13,-28.35,71.71,23.03,92008.0,-50765.0,128306.0,216.00,40.35,31.0,H-SC,2.10,166.0,-0.55,0.94,27.30,XR,NTT,BANKS
61,ROUTE,-2.42,-59.32,313.41,68.16,169636.0,-78940.0,54126.0,2215.45,-67.06,27.0,H-SC,27.44,141.0,-0.47,0.40,0.00,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.86,0.89,20.5,21.57,20264.0,870.0,98850.0,397.04,-12.83,56.0,H-SC,3.97,167.0,0.04,0.72,44.25,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.51,2.17,103.50,107.91,156124.0,3200.0,150844.0,1641.55,-8.17,62.0,M-SC,9.82,220.0,0.02,1.10,13.48,OX40N,ATH,CABLES
66,SIS,-2.15,-26.44,64.85,21.26,52967.0,-29356.0,81676.0,528.00,1923.83,41.0,H-SC,4.06,168.0,-0.55,0.60,12.60,OX40N,NTT,MISC
77,TTKPRESTIG,-0.94,-27.39,37.86,0.09,27702.0,-27607.0,73170.0,770.00,69.90,39.0,M-SC,8.94,250.0,-1.00,0.54,0.70,OX40N,NTT,DURABLES
40,INDIGOPNTS,1.18,-30.06,43.00,0.02,52499.0,-52469.0,122090.0,1408.00,86.38,31.0,M-SC,8.00,222.0,-1.00,0.89,6.20,OX40N,NTT,PAINTS
72,TATAELXSI,-1.18,-31.30,92.18,32.03,83490.0,-41262.0,90573.0,9161.00,-14.15,31.0,H-SC,6.41,158.0,-0.49,0.66,1.45,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,2.86,3.44,15.26,19.23,23751.0,5180.0,155640.0,484.83,24.82,49.0,L-LC,11.35,185.0,0.22,1.14,23.40,XY25,ATH,MINING
26,FINCABLES,-0.51,2.17,103.50,107.91,156124.0,3200.0,150844.0,1641.55,-8.17,62.0,M-SC,9.82,220.0,0.02,1.10,13.48,OX40N,ATH,CABLES
50,MASFIN,-1.86,0.89,20.50,21.57,20264.0,870.0,98850.0,397.04,-12.83,56.0,H-SC,3.97,167.0,0.04,0.72,44.25,XR,ATH,FINANCE
1,ABB,0.77,6.87,36.21,45.57,101243.0,17981.0,279600.0,7934.00,-31.30,70.0,H-MC,2.84,125.0,0.18,2.04,24.16,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.77,6.87,36.21,45.57,101243.0,17981.0,279600.0,7934.00,-31.30,70.0,H-MC,2.84,125.0,0.18,2.04,24.16,AR,NTT,ELECTRICAL
50,MASFIN,-1.86,0.89,20.50,21.57,20264.0,870.0,98850.0,397.04,-12.83,56.0,H-SC,3.97,167.0,0.04,0.72,44.25,XR,ATH,FINANCE
26,FINCABLES,-0.51,2.17,103.50,107.91,156124.0,3200.0,150844.0,1641.55,-8.17,62.0,M-SC,9.82,220.0,0.02,1.10,13.48,OX40N,ATH,CABLES
39,INDIAMART,0.27,-5.20,118.07,106.72,138045.0,-6418.0,116918.0,4810.62,-57.23,48.0,H-SC,7.65,140.0,-0.05,0.85,16.96,AR,ATH,MISC
85,ZYDUSLIFE,0.48,-4.14,41.49,35.63,83349.0,-8685.0,200889.0,1286.17,-16.67,52.0,H-MC,5.75,120.0,-0.10,1.47,11.84,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.96,-8.63,46.92,34.24,134139.0,-26999.0,285889.0,1972.00,-28.16,18.0,X-LC,4.52,3.0,-0.20,2.09,0.00,X40,NTT,IT
33,HCLTECH,-0.15,-5.71,31.33,23.83,71470.0,-13803.0,228121.0,1908.19,-0.74,22.0,X-LC,5.04,8.0,-0.19,1.67,9.45,X40,ATH,IT
73,TCS,-0.39,-26.26,60.78,18.55,169514.0,-99340.0,278897.0,4311.59,-33.46,25.0,X-LC,3.07,2.0,-0.59,2.04,0.00,X40,ATH,IT
8,AWL,0.57,-34.30,139.25,57.19,298378.0,-111852.0,214275.0,485.00,-62.47,28.0,X-SC,17.28,81.0,-0.37,1.57,0.57,XY24,NTT,FMCG
6,ASIANPAINT,0.45,-16.92,45.57,20.95,95322.0,-42592.0,209176.0,3460.25,-13.71,30.0,X-LC,7.37,30.0,-0.45,1.53,12.69,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,0.54,0.73,42.66,43.70,106595.0,1804.0,249872.0,735.00,-8.57,53.0,X-MC,1.80,70.0,0.02,1.83,15.89,XY24,BTT,FMCG
78,UNITDSPR,0.18,3.20,17.02,20.77,41128.0,7499.0,241643.0,1644.00,-8.47,60.0,X-MC,2.37,64.0,0.18,1.77,10.84,X40N,NTT,BREWERIES
65,SIEMENS,0.24,-16.06,49.54,25.51,77381.0,-29895.0,156200.0,4671.50,31.28,52.0,H-LC,2.39,51.0,-0.39,1.14,15.11,AR,ATH,ELECTRICAL
81,VBL,1.07,-8.27,47.91,35.67,138803.0,-26126.0,289716.0,671.64,-18.98,42.0,X-LC,2.83,13.0,-0.19,2.12,4.44,X40N,ATH,FMCG
34,HINDUNILVR,0.29,-10.06,26.38,13.67,60991.0,-25863.0,231200.0,2922.00,-33.31,40.0,X-LC,2.85,9.0,-0.42,1.69,9.15,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.06,-11.29,33.09,18.07,52504.0,-20185.0,158670.0,35195.0,-24.45,35.0,X-MC,10.30,58.0,-0.38,1.16,0.0,X40,ATH,PHARMA
3,ACC,-0.27,-31.39,139.24,64.15,227337.0,-74681.0,163270.0,3906.0,-48.05,37.0,X-SC,8.21,85.0,-0.33,1.19,0.0,XY24,BTT,CEMENT
12,BATAINDIA,-1.61,-45.91,149.97,35.20,104373.0,-59074.0,69596.0,2096.0,-8.74,36.0,X-SC,21.20,91.0,-0.57,0.51,0.0,X40,NTT,FOOTWEAR
42,INFY,-1.96,-8.63,46.92,34.24,134139.0,-26999.0,285889.0,1972.0,-28.16,18.0,X-LC,4.52,3.0,-0.20,2.09,0.0,X40,NTT,IT
58,RELAXO,-2.09,-54.37,233.81,52.31,154857.0,-78928.0,66232.0,1176.0,-51.41,32.0,X-SC,14.25,92.0,-0.51,0.48,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,1.18,-30.08,104.29,42.84,47403.0,-19553.0,45453.0,424.00,-54.27,51.0,X-SC,19.56,80.0,-0.41,0.33,9.33,XY24,NTT,MISC
58,RELAXO,-2.09,-54.37,233.81,52.31,154857.0,-78928.0,66232.0,1176.00,-51.41,32.0,X-SC,14.25,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-1.61,-45.91,149.97,35.20,104373.0,-59074.0,69596.0,2096.00,-8.74,36.0,X-SC,21.20,91.0,-0.57,0.51,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,0.40,-4.51,29.94,24.08,36294.0,-5728.0,121222.0,1486.00,-5.20,53.0,X-SC,11.65,87.0,-0.16,0.89,11.03,XY24,NTT,HEALTHCARE
35,HONAUT,0.32,-22.70,85.41,43.32,107531.0,-36972.0,125900.0,58357.33,-32.13,39.0,X-SC,10.60,90.0,-0.34,0.92,2.22,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.20,-22.18,42.36,10.78,79217.0,-53310.0,187008.0,452.00,-54.31,41.0,X-LC,19.02,1.0,-0.67,1.37,3.00,X40,NTT,FMCG
73,TCS,-0.39,-26.26,60.78,18.55,169514.0,-99340.0,278897.0,4311.59,-33.46,25.0,X-LC,3.07,2.0,-0.59,2.04,0.00,X40,ATH,IT
42,INFY,-1.96,-8.63,46.92,34.24,134139.0,-26999.0,285889.0,1972.00,-28.16,18.0,X-LC,4.52,3.0,-0.20,2.09,0.00,X40,NTT,IT
75,TMPV,-0.79,-22.68,59.05,22.98,103141.0,-51231.0,174667.0,600.00,-82.53,60.0,X-LC,3.25,4.0,-0.50,1.28,11.66,XY24,NTT,AUTO
33,HCLTECH,-0.15,-5.71,31.33,23.83,71470.0,-13803.0,228121.0,1908.19,-0.74,22.0,X-LC,5.04,8.0,-0.19,1.67,9.45,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.27,-31.82,94.91,32.89,49961.0,-24566.0,52640.0,1800.00,-347.90,57.0,L-MC,11.01,258.0,-0.49,0.38,44.97,XR,NTT,BANKS
67,SONACOMS,-0.56,-9.18,53.15,39.09,48832.0,-9284.0,91875.0,804.02,-28.03,61.0,M-MC,5.13,193.0,-0.19,0.67,29.97,AR,ATH,AUTO
50,MASFIN,-1.86,0.89,20.50,21.57,20264.0,870.0,98850.0,397.04,-12.83,56.0,H-SC,3.97,167.0,0.04,0.72,44.25,XR,ATH,FINANCE
79,VAIBHAVGBL,-2.33,-23.68,110.65,60.77,154350.0,-43281.0,139494.0,521.00,60.88,56.0,H-SC,0.80,160.0,-0.28,1.02,26.47,XR,NTT,JEWELLERY
18,COALINDIA,2.86,3.44,15.26,19.23,23751.0,5180.0,155640.0,484.83,24.82,49.0,L-LC,11.35,185.0,0.22,1.14,23.40,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.77,6.87,36.21,45.57,101243.0,17981.0,279600.0,7934.00,-31.30,70.0,H-MC,2.84,125.0,0.18,2.04,24.16,AR,NTT,ELECTRICAL
7,ATULAUTO,0.61,-13.18,67.66,45.57,115804.0,-25976.0,171156.0,844.00,3710.81,67.0,M-SC,5.08,248.0,-0.22,1.25,29.53,XY24,NTT,AUTO
11,BANDHANBNK,1.28,-19.40,138.73,92.42,311472.0,-54042.0,224517.0,400.00,92.53,68.0,H-SC,7.84,174.0,-0.17,1.64,29.75,XY24,NTT,BANKS
79,VAIBHAVGBL,-2.33,-23.68,110.65,60.77,154350.0,-43281.0,139494.0,521.00,60.88,56.0,H-SC,0.80,160.0,-0.28,1.02,26.47,XR,NTT,JEWELLERY
67,SONACOMS,-0.56,-9.18,53.15,39.09,48832.0,-9284.0,91875.0,804.02,-28.03,61.0,M-MC,5.13,193.0,-0.19,0.67,29.97,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.20
1,25,44.66
2,50,76.80


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.32
MC    30.49
LC    23.21
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      23.17
X40N     15.29
AR        9.01
XY25      8.80
XR        8.77
OX40N     7.36
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.08
H-SC    22.69
X-LC    19.85
X-SC    11.89
M-SC    10.93
H-MC     4.96
M-MC     2.07
H-LC     1.14
L-LC     1.14
M-LC     1.08
L-SC     0.81
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.06,-15.04,53.69
FINANCE,13.35,-14.37,59.37
IT,11.55,-35.40,102.96
MISC,7.12,-33.39,87.95
ELECTRICAL,6.27,-9.36,49.27
PAINTS,5.12,-31.83,51.41
INSURANCE,4.88,-1.76,36.58
PHARMA,4.17,-3.19,35.67
AUTO,3.20,-19.76,61.18


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3314856.0,21
AR,1338065.0,10
X40,1331188.0,15
XR,1291336.0,12
X40N,1030599.0,10
OX40N,784776.0,10
XY25,415093.0,6
SR,299773.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3689390.0,24
M-SC,1424323.0,14
X-MC,1359850.0,15
X-SC,1178199.0,10
X-LC,1176559.0,12
H-MC,399558.0,3
L-SC,180572.0,2
M-LC,144071.0,1
M-MC,102071.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1259963.0      6
           AR         952204.0      5
           XR         828067.0      7
M-SC       XY24       825209.0      6
X-SC       XY24       609412.0      4
X-LC       X40        606953.0      6
X-MC       X40        545601.0      7
           X40N       411303.0      4
X-SC       X40N       390153.0      4
H-SC       OX40N      349383.0      4
M-SC       OX40N      343334.0      5
H-SC       SR         299773.0      2
X-LC       X40N       229143.0      2
H-MC       XY24       214966.0      1
X-LC       XY24       211159.0      2
X-MC       XY25       208799.0      2
           XY24       194147.0      2
H-MC       AR         184592.0      2
M-SC       XR         180724.0      2
X-SC       X40        178634.0      2
M-LC       XR         144071.0      1
X-LC       XY25       129304.0      2
L-SC       OX40N       92059.0      1
           XR          88513.0      1
H-LC       AR          77381.0      1
M-SC       AR          75056.0      1
M-MC       XY25        53239.0      1
L-MC       XR          49961.0      1
M-MC       AR          48832.0      1
L-LC       XY25        23751.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
